# Itinerary Basics

Basic report on stage itinerary - stage name, distance etc.

In [1]:
import notebookimport

In [2]:
import os
import sqlite3
import pandas as pd

In [43]:
#!pip3 install tracery
#!pip3 install inflect

In [14]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='spain18.db'
    conn = sqlite3.connect(dbname)
    rally='Spain'
    rebase = 'PAD'
    rebase = ''

In [15]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

def traceryparse(rules, root, modifiers=base_english):
    grammar = tracery.Grammar(rules)
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)
    
    return grammar.flatten(root)
    

## Rally Summary

In [16]:
def dbGetRallySummary(rally):
    q='''
    SELECT o.*, t.totalDistance FROM (SELECT ce.`country.name`, ce.startDate, ce.finishDate, ce.name, ce.`country.iso3`, ce.surfaces,
    COUNT(*) numOfStages, SUM(itc.distance) AS compDistanceKm
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    WHERE ce.`country.name`="{rally}" AND type='StageStart') AS o
    JOIN (SELECT ce.`country.name`, SUM(itc.distance) totalDistance FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    WHERE ce.`country.name`="{rally}") AS t ON o.`country.name` = t.`country.name`
    '''.format(rally=rally)
    
    rallydetails = pd.read_sql(q,conn)
    
    return rallydetails

In [17]:
rs = dbGetRallySummary(rally)
rs

,country.name,startDate,finishDate,name,country.iso3,surfaces,numOfStages,compDistanceKm,totalDistance
0,Spain,2018-10-25,2018-10-28,RallyRACC Catalunya - Rally de España,ESP,,18,331.58,1495.73


In [18]:
rs.dtypes

country.name       object
startDate          object
finishDate         object
name               object
country.iso3       object
surfaces           object
numOfStages         int64
compDistanceKm    float64
totalDistance     float64
dtype: object

In [19]:
rules = {'origin': "#name# (#startDate# to #finishDate#) #stages#. #dist#. #surface#.",
         'stages': "runs over #numOfStages# competitive special stages",
         'dist': "The distance covered on the special stages is #compDistanceKm.round#km, with an overall rally distance of #totalDistance.round#km",
         'surface':"The special stage surface type is predominantly #surfaces#",
        }

In [20]:
rs['report'] = rs.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english, pytracery_logic]), axis=1)

In [21]:
rs[rs['country.name']==rally]['report'].iloc[0]

'RallyRACC Catalunya - Rally de España (2018-10-25 to 2018-10-28) runs over 18 competitive special stages. The distance covered on the special stages is 331.58km, with an overall rally distance of 1495.73km. The special stage surface type is predominantly .'

In [22]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)

In [23]:
README='''# Rally Report - {}

*This report is unofficial and is not associated in any way with the Fédération Internationale de l’Automobile (FIA) or WRC Promoter GmbH.*


{}
'''.format(rally, rs[rs['country.name']==rally]['report'].iloc[0])

with open('{}/README.md'.format(basepath), 'w') as out_file:
    out_file.write(README)

In [24]:
import inflect
p=inflect.engine()

In [25]:
rs['startDate']=pd.to_datetime(rs['startDate'])
rs['startDate'].loc[0].strftime("%A %d,  %B %Y")

'Thursday 25,  October 2018'

In [26]:
p.number_to_words(p.ordinal((int(rs['startDate'].loc[0].strftime("%d")))))

'twenty-fifth'

## Itinerary Items

In [27]:
def dbGetTimeControls(rally):

    q='''
    SELECT il.name AS date, itc.*, ce.timeZoneOffset,
         isc.itinerarySectionId, isc.name AS section, isc.`order`
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally)
    time_controls = pd.read_sql(q,conn)
    time_controls['firstCarDueDateTimeLocal']=pd.to_datetime(time_controls['firstCarDueDateTimeLocal'])
    return time_controls

In [28]:
q='''
    SELECT *
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally)
xx =pd.read_sql(q,conn)
xx.columns

Index(['code', 'controlId', 'controlPenalties', 'distance', 'eventId',
       'firstCarDueDateTime', 'firstCarDueDateTimeLocal', 'location',
       'stageId', 'status', 'targetDuration', 'targetDurationMs',
       'timingPrecision', 'type', 'itineraryLegId',
       'itinerarySections.itinerarySectionId', 'roundingPolicy', 'categories',
       'clerkOfTheCourse', 'country.countryId', 'country.iso2', 'country.iso3',
       'country.name', 'countryId', 'eventId', 'finishDate', 'location',
       'mode', 'name', 'organiserUrl', 'slug', 'startDate', 'stewards',
       'surfaces', 'templateFilename', 'timeZoneId', 'timeZoneName',
       'timeZoneOffset', 'trackingEventId', 'itineraryLegId',
       'itinerarySectionId', 'name', 'order', 'itineraryId', 'itineraryLegId',
       'legDate', 'name', 'order', 'startListId', 'status'],
      dtype='object')

In [29]:
xx.head()

,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,...,itinerarySectionId,name,order,itineraryId,itineraryLegId,legDate,name,order,startListId,status
0,TC0,3410,All,NaN,37,2018-10-25T11:15:00,2018-10-25T13:15:00+02:00,START - Podium SALOU,-1,Completed,...,327,Section 1,1,124,143,2018-10-25,Thursday 25th October,1,236.0,Completed
1,TC0A,3411,Late,111.12,37,2018-10-25T13:15:00,2018-10-25T15:15:00+02:00,Regroup IN - Barcelona,-1,Completed,...,327,Section 1,1,124,143,2018-10-25,Thursday 25th October,1,236.0,Completed
2,TC1,3451,None,NaN,37,2018-10-25T16:05:00,2018-10-25T18:05:00+02:00,Regroup OUT,823,Completed,...,328,Section 2,2,124,143,2018-10-25,Thursday 25th October,1,236.0,Completed
3,SS1,3452,None,3.20,37,2018-10-25T16:08:00,2018-10-25T18:08:00+02:00,Barcelona (asphalt) TV Live,823,Completed,...,328,Section 2,2,124,143,2018-10-25,Thursday 25th October,1,236.0,Completed
4,TC1A,3415,Late,108.06,37,2018-10-25T18:08:00,2018-10-25T20:08:00+02:00,Parc Fermé IN (PortAventura),-1,Completed,...,328,Section 2,2,124,143,2018-10-25,Thursday 25th October,1,236.0,Completed


In [30]:
import datetime
t=datetime.time(abs(int(360/60)), 360 % 60)
dt = datetime.datetime.combine(datetime.date.today(), t)
dt.isoformat()

'2018-10-27T06:00:00'

In [31]:
t = datetime.datetime.combine(datetime.date.today(),datetime.time(0))
(t  + datetime.timedelta( minutes=-359)).isoformat()

'2018-10-26T18:01:00'

In [32]:
xx[:2][['timeZoneId','timeZoneOffset']]

,timeZoneId,timeZoneOffset
0,Romance Standard Time,120
1,Romance Standard Time,120


In [33]:
if __name__=='__main__':
    time_controls = dbGetTimeControls(rally)
    display(time_controls.head())

,date,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,...,targetDurationMs,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,roundingPolicy,timeZoneOffset,itinerarySectionId,section,order
0,Thursday 25th October,TC0,3410,All,NaN,37,2018-10-25T11:15:00,2018-10-25 11:15:00,START - Podium SALOU,-1,...,NaN,Minute,TimeControl,143,327,NoRounding,120,327,Section 1,1
1,Thursday 25th October,TC0A,3411,Late,111.12,37,2018-10-25T13:15:00,2018-10-25 13:15:00,Regroup IN - Barcelona,-1,...,7200000.0,Minute,TimeControl,143,327,NoRounding,120,327,Section 1,1
2,Thursday 25th October,TC1,3451,None,NaN,37,2018-10-25T16:05:00,2018-10-25 16:05:00,Regroup OUT,823,...,10200000.0,Minute,TimeControl,143,328,NoRounding,120,328,Section 2,2
3,Thursday 25th October,SS1,3452,None,3.20,37,2018-10-25T16:08:00,2018-10-25 16:08:00,Barcelona (asphalt) TV Live,823,...,180000.0,Minute,StageStart,143,328,RoundToClosestMinute,120,328,Section 2,2
4,Thursday 25th October,TC1A,3415,Late,108.06,37,2018-10-25T18:08:00,2018-10-25 18:08:00,Parc Fermé IN (PortAventura),-1,...,7200000.0,Minute,TimeControl,143,328,NoRounding,120,328,Section 2,2


In [34]:
#Check datetime type
time_controls['firstCarDueDateTime'] = pd.to_datetime(time_controls['firstCarDueDateTime'])

In [35]:
import datetime

def newtime(row):
    t=datetime.timedelta( minutes=row['timeZoneOffset'])
    return row['firstCarDueDateTime']+ t

time_controls['mylocaltime'] = time_controls.apply(lambda row: newtime(row),axis=1)

In [36]:
rules = {'origin': "#mylocaltime.pdtime(%H:%M:%S)# #code# #location# #distance.isNotNull(post=km).brackets# \[#targetDuration#\]",
        }

In [37]:
dategroups = time_controls.groupby('date',sort=False)
for key in dategroups.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=dategroups.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        g2 = grouped2.get_group(key2)
        l=len(g2[g2['code'].str.startswith('SS')])
        print('{} - {} special {}\n'.format(key2,p.number_to_words(l),p.plural_noun('stage',l)))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Thursday 25th October:

Section 1 - zero special stages

	13:15:00 TC0 START - Podium SALOU  [None]
	15:15:00 TC0A Regroup IN - Barcelona (111.12km) [02:00:00]


Section 2 - one special stage

	18:05:00 TC1 Regroup OUT  [02:50:00]
	18:08:00 SS1 Barcelona (asphalt) TV Live (3.2km) [00:03:00]
	20:08:00 TC1A Parc Fermé IN (PortAventura) (108.06km) [02:00:00]


---

Friday 26th October:

Section 3 - three special stages

	08:00:00 TC1B Parc Fermé OUT - Service IN  [None]
	08:15:00 TC1C Service OUT  [00:15:00]
	09:30:00 TC2 Gandesa (79.93km) [01:15:00]
	09:33:00 SS2 Gandesa 1 (gravel) (7.0km) [00:03:00]
	10:03:00 TC3 Pesells (10.07km) [00:30:00]
	10:06:00 SS3 Pesells 1 (gravel) (26.59km) [00:03:00]
	11:17:00 TC4 La Fatarella (32.48km) [01:11:00]
	11:20:00 SS4 La Fatarella -Vilalba 1 (gravel & asphalt) (38.85km) [00:03:00]
	13:15:00 TC4A Regroup and Technical Zone  IN (85.37km) [01:55:00]


Section 4 - three special stages

	13:35:00 TC4B Regroup OUT - Service IN  [00:20:00]
	14:05:00 T

In [51]:


def initStageReports(rebase='overall'):
    
    sectionREADME_base = '''### {section} Report
    '''

    SUMMARY ='\n'
    
    rules['stages'] = "#code# - #location# #distance.isNotNull(post=km).brackets#"

    dn = '' if not rebase or 'overall' in rebase else '_'+rebase
  
    sections = time_controls.groupby('section',sort=False)
    keyorder = [k for k in time_controls.sort_values('order')['section'].unique() if k in sections.groups.keys()]
    for key in keyorder:

        sectionfn = '{bp}/{key}_report{dn}.md'.format(bp=basepath,key=key, dn=dn)

        with open(sectionfn, 'w') as out_file:
            out_file.write('')

        sectionControls = sections.get_group(key)
        sstages = sectionControls[sectionControls['code'].str.startswith('SS')]
        l=len(sstages)
        title = '# {}, {}\n\nThis section comprises {} special {}'.format(key,sectionControls['date'].iloc[0], p.number_to_words(l),p.plural_noun('stage',l))
        sectionREADME = '''{title}'''.format( title=title)

        sstage=[]
        for r in sstages.apply(lambda row: pandas_row_mapper(row, rules, "#stages#",[base_english,inflect_english]), axis=1):
            sstage.append(r)
        sectionREADME = '{} ({})'.format(sectionREADME,', '.join(sstage))


        sectionREADME = '''{s}\n\nThe full scheduled itinerary for the section was as follows:\n'''.format(s=sectionREADME,)

        controls = []
        for r in sectionControls.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            controls.append(r)
        sectionREADME = '{}\n\t- {}\n'.format(sectionREADME, '\n\t- '.join(controls))

        print(sectionREADME,'\n------\n')
        with open(sectionfn, 'a') as out_file:
            out_file.write(sectionREADME)

        #Add section
        SUMMARY = '{summary}\n* [{key}]({key}_report{dn}.md)\n'.format(summary=SUMMARY, key=key,dn=dn)

        sstageDict = sstages.set_index('code').to_dict(orient='index')
        #Add special stages
        for s in sstageDict: #[Section 2](Section 2_report.md)
            SUMMARY = '{summary}  - [{s} - {n}]({s}_report{dn}.md)\n'.format(summary=SUMMARY,
                                                                              s=s, 
                                                                              n=sstageDict[s]['location'],
                                                                              dn=dn)

    with open('{}/SUMMARY{}.md'.format(basepath,dn), 'a') as out_file:
        out_file.write(SUMMARY)

initStageReports(rebase)
#initStageReports()

# Section 1, Thursday 25th October

This section comprises zero special stages (date, code, controlId, controlPenalties, distance, eventId, firstCarDueDateTime, firstCarDueDateTimeLocal, location, stageId, status, targetDuration, targetDurationMs, timingPrecision, type, itineraryLegId, itinerarySections.itinerarySectionId, roundingPolicy, timeZoneOffset, itinerarySectionId, section, order, mylocaltime)

The full scheduled itinerary for the section was as follows:

	- 13:15:00 TC0 START - Podium SALOU  [None]
	- 15:15:00 TC0A Regroup IN - Barcelona (111.12km) [02:00:00]
 
------

# Section 2, Thursday 25th October

This section comprises one special stage (SS1 - Barcelona (asphalt) TV Live (3.2km))

The full scheduled itinerary for the section was as follows:

	- 18:05:00 TC1 Regroup OUT  [02:50:00]
	- 18:08:00 SS1 Barcelona (asphalt) TV Live (3.2km) [00:03:00]
	- 20:08:00 TC1A Parc Fermé IN (PortAventura) (108.06km) [02:00:00]
 
------

# Section 3, Friday 26th October

This section co

In [49]:
sstages['date']

NameError: name 'sstages' is not defined

In [40]:
sstages.set_index('code').to_dict(orient='index')

NameError: name 'sstages' is not defined

In [41]:
print(SUMMARY)

NameError: name 'SUMMARY' is not defined

In [42]:
!head -n100 "report/Section 3_report.md"

# Section 3, Friday 26th October

This section comprises three special stages (SS2 - Gandesa 1 (gravel) (7.0km), SS3 - Pesells 1 (gravel) (26.59km), SS4 - La Fatarella -Vilalba 1 (gravel & asphalt) (38.85km))

The full scheduled itinerary for the section was as follows:

	- 08:00:00 TC1B Parc Fermé OUT - Service IN  [None]
	- 08:15:00 TC1C Service OUT  [00:15:00]
	- 09:30:00 TC2 Gandesa (79.93km) [01:15:00]
	- 09:33:00 SS2 Gandesa 1 (gravel) (7.0km) [00:03:00]
	- 10:03:00 TC3 Pesells (10.07km) [00:30:00]
	- 10:06:00 SS3 Pesells 1 (gravel) (26.59km) [00:03:00]
	- 11:17:00 TC4 La Fatarella (32.48km) [01:11:00]
	- 11:20:00 SS4 La Fatarella -Vilalba 1 (gravel & asphalt) (38.85km) [00:03:00]
	- 13:15:00 TC4A Regroup and Technical Zone  IN (85.37km) [01:55:00]


In [ ]:
sectionControls['code']

In [ ]:
sectionControls#[sectionControls['code']]

In [ ]:
l